In [1]:
# based on this guide: https://www.tensorflow.org/get_started/eager

from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe

#tf.enable_eager_execution() # must be run on a start up program

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

import datetime
import pandas as pd
import numpy as np
import random


Instructions for updating:
Use the retry module or similar alternatives.
TensorFlow version: 1.7.0
Eager execution: False


In [2]:
# load in the data

metadata_file = '/Users/jaspermeijering/Google Drive/a Study/EPA Study Abroad - Carnegie Mellon University/Courses/CMU - 95845 - Applied Analytics The Machine Learning Pipeline/Machine Learning Pipeline Final Project/Data/FalseNews_Code_Data/data/metadata_anon.txt'

# Read meta data 
fin = open(metadata_file,'r')
lines = fin.readlines()
fin.close()
cascade_id2metadata={}
for line in lines:
    line = line.replace('\n','')
    item = eval(line)
    cascade_id2metadata[item[0]] = item[1]

    # Get static measures
veracity = []
virality = []
depth = []
breadth = []
size = []
verified = []
nfollowers = []
nfollowees = []
engagement = []
category = []
day = []
hour = []
cascade= []

for cascade,metadata in cascade_id2metadata.items():
    veracity.append(metadata['veracity'])
    virality.append(metadata['virality'])
    depth.append(metadata['depth'])
    breadth.append(metadata['max_breadth'])
    size.append(metadata['size'])
    verified.append(metadata['verified_list'][0])
    nfollowers.append(metadata['num_followers_list'][0])
    nfollowees.append(metadata['num_followees_list'][0])
    engagement.append(metadata['engagement_list'][0])
    category.append(metadata['rumor_category'])
    day.append(metadata['start_date'].day)
    hour.append(metadata['start_date'].hour)
    #cascadeID.append(cascade) # for merging models

# Convert to data frame
df = pd.DataFrame({'veracity': veracity, 'virality': virality, 'depth': depth, 'breadth': breadth, 'size': size, 'verified': verified, 'nfollowers': nfollowers, 
                   'nfollowees': nfollowees, 'engangement': engagement, 'category': category, 'day': day, 'hour': hour})

# Inspect
df.head(5)


#def parse_data(line):
 #   example_defaults = [[0.], [0.], [0.], [0.], [0]]  # sets field types
  #features = tf.reshape(parsed_line[:-1], shape=(4,))
   # # Last field is the label
  #  label = tf.reshape(parsed_line[-1], shape=())
  #  return features, label



,breadth,category,day,depth,engangement,hour,nfollowees,nfollowers,size,veracity,verified,virality
0,1,Politics,22,0,7.092405,21,236.0,164.0,1,FALSE,False,NaN
1,1,Politics,23,0,8.394500,14,1999.0,870.0,1,FALSE,False,NaN
2,1,Politics,9,0,6.729679,16,1982.0,452.0,1,FALSE,False,NaN
3,1,Politics,15,0,0.186418,20,55.0,5.0,1,FALSE,False,NaN
4,1,Politics,14,0,1.425915,14,129.0,95.0,1,FALSE,False,NaN


In [3]:
df_saved = df


In [4]:
df = df_saved


### Make data numeric

In [5]:
# Make category numeric
df["category"] = df["category"].astype('category')
df["category_num"] = df.category.cat.rename_categories([1,2,3,4,5,6,7])
df["category_num"] = df["category_num"].astype('int')
print("Transformed categories to numeric values, this is how they match",df.category.unique(),df.category_num.unique())

# Make veracity numeric
df["veracity"] = df["veracity"].astype('category')
df["veracity_num"] = df.veracity.cat.rename_categories([1,2,3])
df["veracity_num"] = df["veracity_num"].astype('int')
print("Transformed veracity to numeric values, this is how they match",df.veracity.unique(),df.veracity_num.unique())

# Make verified numeric
df["verified_cat"] = df["verified"].astype('category')
df["verified_cat"] = df["verified_cat"].cat.set_categories(["True","False","None"])
#df["verified_cat"].isnull().sum() # 136 nan / none values for verified
df.loc[df["verified_cat"].isnull(),'verified'] = "None" # -> leave these cascades in. # this is wrong
df["verified_num"] = df.verified_cat.cat.rename_categories([1,2,3])
df["verified_num"] = df["verified_num"].astype('int')
print("Transformed verified to numeric values, this is how they match",df.verified.unique(),df.verified_num.unique())

#df.dtypes
df.head()


Transformed categories to numeric values, this is how they match [Politics, Viral Photos/Stories/Urban Legends, War/Terrorism/Shootings, Science/Nature/Tech/Food/Health, Entertainment, Business, Natural Disasters]
Categories (7, object): [Politics, Viral Photos/Stories/Urban Legends, War/Terrorism/Shootings, Science/Nature/Tech/Food/Health, Entertainment, Business, Natural Disasters] [4 6 7 5 2 1 3]
Transformed veracity to numeric values, this is how they match [FALSE, TRUE, MIXED]
Categories (3, object): [FALSE, TRUE, MIXED] [1 3 2]
Transformed verified to numeric values, this is how they match ['None'] [-9223372036854775808]


,breadth,category,day,depth,engangement,hour,nfollowees,nfollowers,size,veracity,verified,virality,category_num,veracity_num,verified_cat,verified_num
0,1,Politics,22,0,7.092405,21,236.0,164.0,1,FALSE,None,NaN,4,1,NaN,-9223372036854775808
1,1,Politics,23,0,8.394500,14,1999.0,870.0,1,FALSE,None,NaN,4,1,NaN,-9223372036854775808
2,1,Politics,9,0,6.729679,16,1982.0,452.0,1,FALSE,None,NaN,4,1,NaN,-9223372036854775808
3,1,Politics,15,0,0.186418,20,55.0,5.0,1,FALSE,None,NaN,4,1,NaN,-9223372036854775808
4,1,Politics,14,0,1.425915,14,129.0,95.0,1,FALSE,None,NaN,4,1,NaN,-9223372036854775808


# verified is wrong

### Drop NaN values for virality. 

In [6]:
print(df.virality.isnull().sum(),"of",len(df),"cascades are single tweet cascades. These tweets were not retweeded. \
We are only interessted in rumours that have been spread and want to predict the veracity based on how information deffused \
These sigle tweets will therefor be dropped from the dataset.")

df = df[np.isfinite(df['virality'])]
df.head(10)

84220 of 126301 cascades are single tweet cascades. These tweets were not retweeded. We are only interessted in rumours that have been spread and want to predict the veracity based on how information deffused These sigle tweets will therefor be dropped from the dataset.


,breadth,category,day,depth,engangement,hour,nfollowees,nfollowers,size,veracity,verified,virality,category_num,veracity_num,verified_cat,verified_num
7777,10703,Viral Photos/Stories/Urban Legends,28,11,25.799399,9,186.0,672.0,23228,MIXED,None,4.003857,6,2,NaN,-9223372036854775808
7778,11783,Science/Nature/Tech/Food/Health,1,9,10.811974,6,313.0,380.0,14827,MIXED,None,2.535338,5,2,NaN,-9223372036854775808
7782,6504,Viral Photos/Stories/Urban Legends,27,13,15.395237,21,518.0,504.0,14129,MIXED,None,4.019705,6,2,NaN,-9223372036854775808
7783,5772,Viral Photos/Stories/Urban Legends,22,8,3.140842,4,189.0,228.0,9972,MIXED,None,3.271008,6,2,NaN,-9223372036854775808
7784,6041,Viral Photos/Stories/Urban Legends,21,8,5.160261,23,174.0,110.0,9526,MIXED,None,3.115942,6,2,NaN,-9223372036854775808
7785,6160,Entertainment,10,8,11.310233,0,846.0,842.0,9124,MIXED,None,2.971147,2,2,NaN,-9223372036854775808
7786,2110,Viral Photos/Stories/Urban Legends,28,14,20.554899,6,542.0,716.0,6203,MIXED,None,5.110521,6,2,NaN,-9223372036854775808
7787,3092,Viral Photos/Stories/Urban Legends,13,10,4.256828,4,364.0,179.0,5882,MIXED,None,3.633757,6,2,NaN,-9223372036854775808
7788,4971,Politics,29,3,4.325858,15,417.0,637.0,5075,MIXED,None,2.041730,4,2,NaN,-9223372036854775808
7789,3374,Viral Photos/Stories/Urban Legends,11,5,29.216077,13,910.0,1140.0,4305,MIXED,None,2.584301,6,2,NaN,-9223372036854775808


### Only keep relevant columns

In [7]:
# Only keep relevant columns
column_names_df = ['veracity_num','breadth', 'category_num', 'depth', 'engangement', 'day','nfollowees', 'nfollowers', 'size', 'verified_num', 'virality']
column_names_x =['breadth', 'category_num', 'depth', 'engangement', 'day','nfollowees', 'nfollowers', 'size', 'verified_num', 'virality']
column_names_x_cat = ['category_num','day','verified_num',]
column_names_x_con = ['breadth', 'depth', 'engangement', 'nfollowees', 'nfollowers', 'size', 'virality']
column_names_y =['veracity_num']

df = df.loc[:,column_names_df]
#df.head()


In [8]:
#set seed
random_array = np.random.choice(["train", "test"], size= len(df), p=[1./2, 1./2])
df["train_test"] = random_array
df.head(10)

,veracity_num,breadth,category_num,depth,engangement,day,nfollowees,nfollowers,size,verified_num,virality,train_test
7777,2,10703,6,11,25.799399,28,186.0,672.0,23228,-9223372036854775808,4.003857,test
7778,2,11783,5,9,10.811974,1,313.0,380.0,14827,-9223372036854775808,2.535338,train
7782,2,6504,6,13,15.395237,27,518.0,504.0,14129,-9223372036854775808,4.019705,train
7783,2,5772,6,8,3.140842,22,189.0,228.0,9972,-9223372036854775808,3.271008,test
7784,2,6041,6,8,5.160261,21,174.0,110.0,9526,-9223372036854775808,3.115942,train
7785,2,6160,2,8,11.310233,10,846.0,842.0,9124,-9223372036854775808,2.971147,test
7786,2,2110,6,14,20.554899,28,542.0,716.0,6203,-9223372036854775808,5.110521,test
7787,2,3092,6,10,4.256828,13,364.0,179.0,5882,-9223372036854775808,3.633757,test
7788,2,4971,4,3,4.325858,29,417.0,637.0,5075,-9223372036854775808,2.041730,train
7789,2,3374,6,5,29.216077,11,910.0,1140.0,4305,-9223372036854775808,2.584301,test


## Running the models

In [9]:
X_train_cat  = df.loc[df["train_test"] == "train",column_names_x_cat ]
X_test_cat  = df.loc[df["train_test"] == "test",column_names_x_cat ]
X_train_con  = df.loc[df["train_test"] == "train",column_names_x_con ]
X_test_con  = df.loc[df["train_test"] == "test",column_names_x_con ]
Y_train = df.loc[df["train_test"] == "train",column_names_y ]
Y_test = df.loc[df["train_test"] == "test",column_names_y ]



In [10]:
def change_format2(X_train_cat):
    X_train_cat.index = np.arange(1, len(X_train_cat) + 1)
    for a in range(7):
        X_train_cat["category_num_%d"% (a+1)] = 0
    for b in range(31):
        X_train_cat["day_%d"% (b+1)] = 0
    for c in range(3):
        X_train_cat["verified_%d"% (c+1)] = 0
    
    for i in range(len(X_train_cat)):
        for j in range(8):
            if X_train_cat.iloc[i,0] == j:
                X_train_cat["category_num_%d"% (j)][i+1] = 1
                break
        for k in range(32):
            if X_train_cat.iloc[i,1] == k:
                X_train_cat["day_%d"% (k)][i+1] = 1
                break
        for l in range(4):
            if X_train_cat.iloc[i,2] == l:
                X_train_cat["verified_%d"% (l)][i+1] = 1
                break     
    return X_train_cat

In [11]:
X_train_cat2 = change_format2(X_train_cat)
X_test_cat2 = change_format2(X_test_cat)


In [12]:
X_train_cat2 = X_train_cat2.iloc[:,3:]
X_test_cat2 = X_test_cat2.iloc[:,3:]
X_test_cat2.head()

,category_num_1,category_num_2,category_num_3,category_num_4,category_num_5,category_num_6,category_num_7,day_1,day_2,day_3,...,day_25,day_26,day_27,day_28,day_29,day_30,day_31,verified_1,verified_2,verified_3
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
X_train_cat  = X_train_cat2.as_matrix()
X_test_cat = X_test_cat2.as_matrix() 
X_train_con  = X_train_con.as_matrix()
X_test_con = X_test_con.as_matrix()
Y_train = Y_train.as_matrix()
Y_test = Y_test.as_matrix()



In [14]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import keras


Using TensorFlow backend.


In [15]:
model_cat = Sequential()
model_cat.add(Dense(4,input_dim = 41)) # or 38?
model_cat.add(Activation('relu'))
model_cat.add(Dense(4))
model_cat.add(Activation('softmax'))

model_cat.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
model_cat.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 168       
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 20        
_________________________________________________________________
activation_2 (Activation)    (None, 4)                 0         
Total params: 188
Trainable params: 188
Non-trainable params: 0
_________________________________________________________________


In [17]:
import pydot
import graphviz

In [18]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

#SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [19]:
model_cat.fit(X_train_cat, keras.utils.to_categorical(Y_train, num_classes=None),
          epochs=20,
          batch_size=128)

Epoch 1/20
21006/21006 [==============================] - 0s 19us/step - loss: 1.1264 - acc: 0.6171
Epoch 2/20
21006/21006 [==============================] - 0s 11us/step - loss: 0.7939 - acc: 0.7588
Epoch 3/20
21006/21006 [==============================] - 0s 12us/step - loss: 0.7024 - acc: 0.7588
Epoch 4/20
21006/21006 [==============================] - 0s 11us/step - loss: 0.6672 - acc: 0.7588
Epoch 5/20
21006/21006 [==============================] - 0s 11us/step - loss: 0.6463 - acc: 0.7564
Epoch 6/20
21006/21006 [==============================] - 0s 11us/step - loss: 0.6336 - acc: 0.7559
Epoch 7/20
21006/21006 [==============================] - 0s 10us/step - loss: 0.6258 - acc: 0.7545
Epoch 8/20
21006/21006 [==============================] - 0s 10us/step - loss: 0.6208 - acc: 0.7584
Epoch 9/20
21006/21006 [==============================] - 0s 10us/step - loss: 0.6177 - acc: 0.7582: 0s - loss: 0.6175 - acc: 0.75
Epoch 10/20
21006/21006 [==============================] - 0s 10us/st

In [20]:
score = model_cat.evaluate(X_test_cat2, keras.utils.to_categorical(Y_test, num_classes=None), batch_size=128)
score

21075/21075 [==============================] - 0s 6us/step


[0.61530835017049301, 0.76132858833242845]

### Categorical model in way that can be combined with other models
Also see: https://keras.io/getting-started/functional-api-guide/

In [21]:

from keras.layers import Input, Dense
from keras.models import Model
# This returns a tensor
inputs = Input(shape=(41,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(4, activation='relu')(inputs)
x = Dense(4, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train_cat, keras.utils.to_categorical(Y_train, num_classes=None),epochs=40) 

Epoch 1/40
21006/21006 [==============================] - 1s 64us/step - loss: 0.9874 - acc: 0.7082
Epoch 2/40
21006/21006 [==============================] - 1s 54us/step - loss: 0.7170 - acc: 0.7588
Epoch 3/40
21006/21006 [==============================] - 1s 43us/step - loss: 0.6680 - acc: 0.7588
Epoch 4/40
21006/21006 [==============================] - 1s 34us/step - loss: 0.6527 - acc: 0.7588
Epoch 5/40
21006/21006 [==============================] - 1s 31us/step - loss: 0.6462 - acc: 0.7588
Epoch 6/40
21006/21006 [==============================] - 1s 32us/step - loss: 0.6411 - acc: 0.7588
Epoch 7/40
21006/21006 [==============================] - 1s 33us/step - loss: 0.6367 - acc: 0.7586
Epoch 8/40
21006/21006 [==============================] - 1s 34us/step - loss: 0.6320 - acc: 0.7636
Epoch 9/40
21006/21006 [==============================] - 1s 44us/step - loss: 0.6269 - acc: 0.7687
Epoch 10/40
21006/21006 [==============================] - 1s 38us/step - loss: 0.6226 - acc: 0.7701

## Continious model

In [22]:
model_con = Sequential()
model_con.add(Dense(4,input_dim = 7)) 
model_con.add(Activation('relu'))
model_con.add(Dense(4))
model_con.add(Activation('softmax'))

model_con.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['accuracy'])

In [23]:
model_con.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 4)                 32        
_________________________________________________________________
activation_3 (Activation)    (None, 4)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 20        
_________________________________________________________________
activation_4 (Activation)    (None, 4)                 0         
Total params: 52
Trainable params: 52
Non-trainable params: 0
_________________________________________________________________


In [24]:
model_con.fit(X_train_con, keras.utils.to_categorical(Y_train, num_classes=None),
          epochs=20,
          batch_size=128
         )

Epoch 1/20
21006/21006 [==============================] - 0s 13us/step - loss: nan - acc: 0.0071
Epoch 2/20
21006/21006 [==============================] - 0s 8us/step - loss: nan - acc: 0.0000e+00
Epoch 3/20
21006/21006 [==============================] - 0s 7us/step - loss: nan - acc: 0.0000e+00
Epoch 4/20
21006/21006 [==============================] - 0s 7us/step - loss: nan - acc: 0.0000e+00
Epoch 5/20
21006/21006 [==============================] - 0s 7us/step - loss: nan - acc: 0.0000e+00
Epoch 6/20
21006/21006 [==============================] - 0s 8us/step - loss: nan - acc: 0.0000e+00
Epoch 7/20
21006/21006 [==============================] - 0s 7us/step - loss: nan - acc: 0.0000e+00
Epoch 8/20
21006/21006 [==============================] - 0s 8us/step - loss: nan - acc: 0.0000e+00
Epoch 9/20
21006/21006 [==============================] - 0s 8us/step - loss: nan - acc: 0.0000e+00
Epoch 10/20
21006/21006 [==============================] - 0s 7us/step - loss: nan - acc: 0.0000e+00
Ep

In [25]:
score_con = model_con.evaluate(X_test_con, keras.utils.to_categorical(Y_test, num_classes=None), batch_size=128)
score_con

21075/21075 [==============================] - 0s 5us/step


[nan, 0.0]

In [32]:
X_train_con.shape

(21006, 7)

In [47]:
#main_input = Input(shape=(100,), dtype='int32', name='main_input') #Example
#main_input = Input(input_dim=7, dtype='int32', name='Categorial_input') # not working
cat_input = Input(shape=(41,), name='Categorial_input') # not sure whether this is the good shape(X_train_con.shape)
con_input = Input(shape=(7,), name='Continous_input')
#lstm_input = ??

x = keras.layers.concatenate([cat_input, con_input])
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

main_output = Dense(4, activation='sigmoid', name='main_output')(x) # from example # 1 or 4

In [48]:
model = Model(inputs=[cat_input, con_input], outputs=[main_output])

In [49]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Categorial_input (InputLayer)   (None, 41)           0                                            
__________________________________________________________________________________________________
Continous_input (InputLayer)    (None, 7)            0                                            
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 48)           0           Categorial_input[0][0]           
                                                                 Continous_input[0][0]            
__________________________________________________________________________________________________
dense_23 (Dense)                (None, 64)           3136        concatenate_5[0][0]              
__________

In [50]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy') # but you need multiple loss function don't you?

In [51]:
model.fit([X_train_cat, X_train_con], keras.utils.to_categorical(Y_train, num_classes=None),
          epochs=50, batch_size=32)

Epoch 1/50
21006/21006 [==============================] - 2s 72us/step - loss: nan
Epoch 2/50
21006/21006 [==============================] - 1s 44us/step - loss: nan
Epoch 3/50
21006/21006 [==============================] - 1s 43us/step - loss: nan
Epoch 4/50
21006/21006 [==============================] - 1s 42us/step - loss: nan
Epoch 5/50
21006/21006 [==============================] - 1s 43us/step - loss: nan
Epoch 6/50
21006/21006 [==============================] - 1s 44us/step - loss: nan
Epoch 7/50
21006/21006 [==============================] - 1s 49us/step - loss: nan
Epoch 8/50
21006/21006 [==============================] - 1s 49us/step - loss: nan
Epoch 9/50
21006/21006 [==============================] - 1s 51us/step - loss: nan
Epoch 10/50
21006/21006 [==============================] - 1s 59us/step - loss: nan
Epoch 11/50
21006/21006 [==============================] - 1s 51us/step - loss: nan
Epoch 12/50
21006/21006 [==============================] - 1s 50us/step - loss: nan
E

In [ ]:
model_total = Sequential()
model_total.add(Dense(4,input_dim = 7)) 
model_total.add(Activation('relu'))
model_total.add(Dense(4))
model_total.add(Activation('softmax'))

model_total.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['accuracy'])


In [27]:
dfg

NameError: name 'dfg' is not defined

# For later:

### Stacked LSTM for sequence classification
In this model, we stack 3 LSTM layers on top of each other, making the model capable of learning higher-level temporal representations.

The first two LSTMs return their full output sequences, but the last one only returns the last step in its output sequence, thus dropping the temporal dimension (i.e. converting the input sequence into a single vector).



In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 16
timesteps = 8
num_classes = 10

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Generate dummy training data
x_train = np.random.random((1000, timesteps, data_dim))
y_train = np.random.random((1000, num_classes))

# Generate dummy validation data
x_val = np.random.random((100, timesteps, data_dim))
y_val = np.random.random((100, num_classes))

model.fit(x_train, y_train,
          batch_size=64, epochs=5,
          validation_data=(x_val, y_val))

